In [101]:
from __future__ import annotations

import warnings
from time import perf_counter

import numpy as np
import pandas as pd
from ase import build
from ase.visualize import view

import matplotlib.pyplot as plt
# from mp_api.client import MPRester
from tqdm import tqdm

from pymatgen.core import Structure, Molecule
from matcalc import (
    AdsorptionCalc,
    SurfaceCalc,
    PESCalculator,
    RelaxCalc
)

warnings.filterwarnings("ignore", category=UserWarning, module="matgl")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="spglib")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
calc = PESCalculator.load_universal("M3GNet")

/Users/keith/miniconda3/envs/matcalc-dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sc = SurfaceCalc(
    calculator=calc,
    relax_bulk=True,
    relax_slab=True,
    fmax=0.1,
    optimizer="BFGS",
    max_steps=200,
    relax_calc_kwargs={},
)

In [5]:
# Build BCC iron conventional unit cell
bulk = Structure(
    lattice=[[3.615, 0.0, 0.0], [0.0, 3.615, 0.0], [0.0, 0.0, 3.615]],
    species=["Li", "Li"],
    coords=[[0, 0, 0], [0.5, 0.5, 0.5]],
)

# First calculate slab energies for 110 surface
miller_index = (1, 1, 0)
slab_results = sc.calc_slabs(
    bulk=bulk,
    miller_indices=miller_index,
    min_slab_size=10.0,
    min_vacuum_size=15.0,
    symmetrize=True,
    inplane_supercell=(2, 2),
    slab_gen_kwargs={
        "center_slab": True,
        "max_normal_search": 1,
    },
    get_slabs_kwargs={
        "tol": 0.1,
    },
)

In [6]:
# Then calculate surface energies
for slab in slab_results:
    surf_energy_results = sc.calc(slab)
    print(f"Structure: {slab['miller_index']}")
    print(f'Surface energy (J/m^2): {surf_energy_results["surface_energy"]:.4f}')
    print('-'*30)

Structure: (1, 0, 0)
Surface energy (J/m^2): 0.0271
------------------------------


In [94]:
ac = AdsorptionCalc(
    calculator=calc,
    relax_slab=True,
    relax_bulk=True,
    relax_adsorbate=True,
    fmax=0.1,
    optimizer="BFGS",
    max_steps=200,
    relax_calc_kwargs={},
)

In [107]:
ase_slab = build.fcc111("Pt", size=(2, 2, 4), vacuum=15.0)
ase_adsorbate = build.molecule("H2")
ase_adslab = ase_slab.copy()
build.add_adsorbate(ase_adslab, ase_adsorbate, height=1.5, position="ontop")

results = ac.calc(
    structure={
        "slab_energy_per_atom": -2.867839813232422,
        "slab": ase_slab,
        "adsorbate_energy": -6.730739593505859,
        "adslab": ase_adslab,
        # "adsorbate": Molecule("HH", [[0, 0, 0], [0, 0, 0.74]]),
        "adsorbate": ase_adsorbate,
    }
)
results

{'slab_energy_per_atom': -2.867839813232422,
 'slab': Atoms(symbols='Pt16', pbc=[True, True, False], cell=[[5.5437171645025325, 0.0, 0.0], [2.7718585822512662, 4.800999895855028, 0.0], [0.0, 0.0, 36.789639165669996]], tags=...),
 'adsorbate_energy': -6.730739593505859,
 'adslab': Atoms(symbols='Pt16H2', pbc=[True, True, False], cell=[[5.5437171645025325, 0.0, 0.0], [2.7718585822512662, 4.800999895855028, 0.0], [0.0, 0.0, 36.789639165669996]], tags=..., calculator=PESCalculator(...)),
 'adsorbate': Atoms(symbols='H2', pbc=False),
 'final_adslab': Structure Summary
 Lattice
     abc : 5.5437171645025325 5.543717164502532 36.789639165669996
  angles : 90.0 90.0 59.99999999999999
  volume : 979.1704298827916
       A : 5.5437171645025325 0.0 0.0
       B : 2.7718585822512662 4.800999895855028 0.0
       C : 0.0 0.0 36.789639165669996
     pbc : True True False
 PeriodicSite: Pt (0.3878, 0.2342, 15.33) [0.04556, 0.04878, 0.4167]
 PeriodicSite: Pt (2.864, 0.3006, 15.17) [0.4853, 0.06262, 0.4

In [ ]:
adslab_atoms = AseAtomsAdaptor.get_atoms(adslabs[0]['final_adslab'])
view(adslab_atoms, viewer='x3d')

In [249]:
bulk = build.bulk('Pt', cubic=True)
# adsorbate = build.molecule("H2O")
adsorbate = Molecule("COO", [[0, 0, 0], [0, 0, 1.16], [0, 0, -1.16]])

ac = AdsorptionCalc(
    calculator=calc,
    relax_slab=True,
    relax_bulk=True,
    relax_adsorbate=False,
    fmax=0.1,
    optimizer="BFGS",
    max_steps=200,
    relax_calc_kwargs={},
)

adslabs = ac.calc_adslabs(
    adsorbate=adsorbate,
    adsorbate_energy=1.0,
    bulk=bulk,
    miller_index=(1,1,1),
    min_slab_size=10.0,
    min_vacuum_size=10.0,
    inplane_supercell=(3, 3),
    slab_gen_kwargs={
        # "center_slab": True,
        # "max_normal_search": 1,
    },
    get_slabs_kwargs={
        # "tol": 0.1,
    },
    adsorption_sites="all",
    height=0.9,
    fixed_height=4.0,
    find_adsorption_sites_args={
        #
    }
    # dry_run=True,
)

In [252]:
[adslabs['adsorbate_energy'] for adslabs in adslabs]

[1.0, 1.0, 1.0, 1.0]

In [174]:
adslab_atoms = AseAtomsAdaptor.get_atoms(adslabs[0]['final_adslab'])
view(adslab_atoms)

<Popen: returncode: None args: ['/Users/keith/miniconda3/envs/matcalc-dev/bi...>

In [23]:
for adslab in adslabs:
    print(f'Natoms: {len(adslab["adslab"])}')
    print(f'Site: {adslab["adsorption_site"]}')
    print(f'Shift: {adslab["shift"]}')
    print(f'Coord {adslab["ads_coord"]}')
    print(f'Miller_index: {adslab["miller_index"]}')
    print(f'Slab: {adslab["slab"].formula} {adslab["slab"].surface_area} {adslab["slab"].shift}')
    print('-'*30)

Natoms: 218
Site: ontop
Shift: 0.5
Coord [ 2.08712122  0.73790879 16.05905058]
Miller_index: (1, 1, 0)
Slab: Li36 55.44378309342945 0.5
------------------------------
Natoms: 218
Site: bridge
Shift: 0.5
Coord [ 1.56534092  2.21372636 16.05905058]
Miller_index: (1, 1, 0)
Slab: Li36 55.44378309342945 0.5
------------------------------
Natoms: 218
Site: bridge
Shift: 0.5
Coord [9.87758345e-16 2.21372636e+00 1.60590506e+01]
Miller_index: (1, 1, 0)
Slab: Li36 55.44378309342945 0.5
------------------------------
Natoms: 218
Site: bridge
Shift: 0.5
Coord [ 0.52178031  0.73790879 16.05905058]
Miller_index: (1, 1, 0)
Slab: Li36 55.44378309342945 0.5
------------------------------
Natoms: 218
Site: hollow
Shift: 0.5
Coord [ 0.69570707  1.72178717 16.05905058]
Miller_index: (1, 1, 0)
Slab: Li36 55.44378309342945 0.5
------------------------------
Natoms: 218
Site: all
Shift: 0.5
Coord [ 2.08712122  0.73790879 16.05905058]
Miller_index: (1, 1, 0)
Slab: Li36 55.44378309342945 0.5
------------------

In [76]:
rc = RelaxCalc(
    calculator=calc,
    fmax=0.1,
    optimizer="BFGS",
    max_steps=200,
)
rc.calc(structure=bulk).keys()

dict_keys(['final_structure', 'energy', 'forces', 'stress', 'a', 'b', 'c', 'alpha', 'beta', 'gamma', 'volume'])

In [41]:
from pymatgen.io.ase import AseAtomsAdaptor
co2 = Molecule("CO2", [[0, 0, 0], [0, 0, 1.16], [0, 0, -1.16]])
co2_atoms = AseAtomsAdaptor.get_atoms(co2)
co2_atoms.set_cell([20, 20, 20])
co2 = rc.calc(co2_atoms)["final_structure"]
co2 = Molecule.from_sites(co2)
# co2 = Molecule.from_sites(AseAtomsAdaptor.get_structure(co2_atoms))
# co2
# co2_atoms
co2

Molecule Summary
Site: C (0.0000, 0.0000, 0.0500)
Site: O (0.0000, 0.0000, 1.1984)
Site: He (-0.0000, -0.0000, -1.2484)

In [197]:
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
asf = AdsorbateSiteFinder(slab['slab'], height=0.9)
adsite = asf.find_adsorption_sites()['all'][0]
asf.add_adsorbate(co2, adsite)

Structure Summary
Lattice
    abc : 7.230000000000001 7.230000000000001 28.920000000000005
 angles : 90.0 90.0 90.0
 volume : 1511.7322680000009
      A : 7.230000000000001 0.0 4.427098178917683e-16
      B : -4.427098178917683e-16 7.230000000000001 4.427098178917683e-16
      C : 0.0 0.0 28.920000000000005
    pbc : True True True
PeriodicSite: Li (0.0, 0.0, 11.75) [0.0, 0.0, 0.4063]
PeriodicSite: Li (-2.214e-16, 3.615, 11.75) [3.425e-33, 0.5, 0.4063]
PeriodicSite: Li (3.615, 0.0, 11.75) [0.5, 0.0, 0.4063]
PeriodicSite: Li (3.615, 3.615, 11.75) [0.5, 0.5, 0.4063]
PeriodicSite: Li (1.808, 1.808, 9.941) [0.25, 0.25, 0.3438]
PeriodicSite: Li (1.808, 5.423, 9.941) [0.25, 0.75, 0.3438]
PeriodicSite: Li (5.423, 1.808, 9.941) [0.75, 0.25, 0.3438]
PeriodicSite: Li (5.423, 5.423, 9.941) [0.75, 0.75, 0.3438]
PeriodicSite: Li (0.0, 0.0, 15.36) [0.0, 0.0, 0.5312]
PeriodicSite: Li (-2.214e-16, 3.615, 15.36) [3.425e-33, 0.5, 0.5312]
PeriodicSite: Li (3.615, 0.0, 15.36) [0.5, 0.0, 0.5312]
PeriodicSi

In [52]:

co2.cart_coords, np.max(co2.cart_coords, axis=0) - np.min(co2.cart_coords, axis=0) + 10

(array([[ 6.59493087e-10,  5.39919175e-11,  5.00437483e-02],
        [ 1.61468758e-08,  1.32446974e-09,  1.19837026e+00],
        [-1.68063507e-08, -1.37855318e-09, -1.24841402e+00]]),
 array([10.00000003, 10.        , 12.44678428]))

In [ ]:
for site in slab["slab"]:
    site.properties["selective_dynamics"] = 
    print(site.properties[])

[ 0.       0.      11.74875]
[-2.21354909e-16  3.61500000e+00  1.17487500e+01]
[ 3.615    0.      11.74875]
[ 3.615    3.615   11.74875]
[1.8075  1.8075  9.94125]
[1.8075  5.4225  9.94125]
[5.4225  1.8075  9.94125]
[5.4225  5.4225  9.94125]
[ 0.       0.      15.36375]
[-2.21354909e-16  3.61500000e+00  1.53637500e+01]
[ 3.615    0.      15.36375]
[ 3.615    3.615   15.36375]
[ 1.8075   1.8075  13.55625]
[ 1.8075   5.4225  13.55625]
[ 5.4225   1.8075  13.55625]
[ 5.4225   5.4225  13.55625]
[ 0.       0.      18.97875]
[-2.21354909e-16  3.61500000e+00  1.89787500e+01]
[ 3.615    0.      18.97875]
[ 3.615    3.615   18.97875]
[ 1.8075   1.8075  17.17125]
[ 1.8075   5.4225  17.17125]
[ 5.4225   1.8075  17.17125]
[ 5.4225   5.4225  17.17125]


In [180]:
from pymatgen.core.surface import SlabGenerator
from pymatgen.util.testing import PymatgenTest
slabgen = SlabGenerator(PymatgenTest.get_structure('Si'), (1,1,1), 10, 10)
slabgen.get_slabs()

[Structure Summary
 Lattice
     abc : 3.840198994394244 5.430860000048243 46.082375204399995
  angles : 134.9999999998674 119.99999086396993 89.99999999983875
  volume : 480.5375357310191
       A : 3.3257101908999998 0.0 -1.9200989669000004
       B : -2.217138494299999 3.1355090603000004 -3.8401979336999994
       C : 0.0 0.0 46.082375204399995
     pbc : True True True
 PeriodicSite: Si (0.0, 0.0, 1.92) [0.0, 0.0, 0.04167]
 PeriodicSite: Si (1.225e-06, 2.352, -1.92) [0.5, 0.75, 0.04167]
 PeriodicSite: Si (0.0, 0.0, 5.76) [0.0, 0.0, 0.125]
 PeriodicSite: Si (1.225e-06, 2.352, 1.92) [0.5, 0.75, 0.125]
 PeriodicSite: Si (0.0, 0.0, 9.6) [0.0, 0.0, 0.2083]
 PeriodicSite: Si (1.225e-06, 2.352, 5.76) [0.5, 0.75, 0.2083]
 PeriodicSite: Si (0.0, 0.0, 13.44) [0.0, 0.0, 0.2917]
 PeriodicSite: Si (1.225e-06, 2.352, 9.6) [0.5, 0.75, 0.2917]
 PeriodicSite: Si (0.0, 0.0, 17.28) [0.0, 0.0, 0.375]
 PeriodicSite: Si (1.225e-06, 2.352, 13.44) [0.5, 0.75, 0.375]
 PeriodicSite: Si (0.0, 0.0, 21.12) [0.

In [186]:
from pymatgen.core import Lattice
from pymatgen.io.ase import AseAtomsAdaptor as AAA

structure = Structure(Lattice.cubic(3.924), ["Pt"]*4,
    [[0,0,0],[0,0.5,0.5],[0.5,0,0.5],[0.5,0.5,0]])
view(AAA.get_atoms(structure))

<Popen: returncode: None args: ['/Users/keith/miniconda3/envs/matcalc-dev/bi...>